# Result analysis

In [ ]:
from dotenv import load_dotenv
import os

# Get the path to the root directory of the repository
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Load environment variables from .env file in the root directory
load_dotenv(os.path.join(root_dir, '.env'))

## Import data

### Load experiment results

In [ ]:
RESULTS_DIR = os.environ.get("RESULTS_PATH")
EXPERIMENT_DIR = None
print(f"The results are stored in the directory {RESULTS_DIR}")

In [ ]:
experiment_batches = [d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, d))]

for i, dir in enumerate(experiment_batches, 1):
    print(f"{i}) {dir}")

In [ ]:
experiment_id = input("Choose the experiment ID (the number)")

try:
    id = int(experiment_id) - 1
    if id < 0 or id >= len(experiment_batches):
        print(f"Error: Invalid experiment ID. Please choose a number between 1 and {len(experiment_batches)}")
        id = 0
        print(f"Using first experiment: {experiment_batches[id]}") 
except ValueError:
    print(f"Error: Invalid input. Please enter a number between 1 and {len(experiment_batches)}")
    id = 0
    print(f"Using first experiment: {experiment_batches[id]}")

exp_folder_name = experiment_batches[id]
EXPERIMENT_DIR = os.path.join(RESULTS_DIR,exp_folder_name)

print(f"The experiment batch selected is {EXPERIMENT_DIR}")

In [ ]:
exp_results_files = [f for f in os.listdir(EXPERIMENT_DIR) if f.endswith('.json')]
print(f"Found {len(exp_results_files)} experiments:")
for file in exp_results_files:
    print(f"  - {file}")

In [ ]:
import json

experiment = exp_results_files[0]
with open(os.path.join(EXPERIMENT_DIR,experiment), 'r') as file:
    data = json.load(file)

In [ ]:
import pandas as pd
import json

experiments_df = pd.DataFrame()

for experiment in exp_results_files:
    try:
        with open(os.path.join(EXPERIMENT_DIR, experiment), 'r') as file:
            data = json.load(file)

            localization = data.get("final_report", {}).get("localization", [])
            if isinstance(localization, list):
                localization_str = ", ".join(localization)
            else:
                localization_str = None
    
        record = {
            "experiment_file": experiment,
            "agent_id" : data.get("agent_id", None),
            "agent_conf_name" : data.get("agent_configuration_name", None),
            "scenario": data.get("app_name", None),
            "fault_name": data.get("testbed", {}).get("fault_name", None),
            "target_namespace": data.get("target_namespace", None),
            "trace_service_starting_point": data.get("trace_service_starting_point", None),
            "rca_tasks_per_iteration": data.get("testbed", {}).get("rca_tasks_per_iteration", 0),
            "max_tool_calls": data.get("testbed", {}).get("max_tool_calls", 0),
            "execution_time_seconds": data.get("stats", {}).get("execution_time_seconds", 0),
            "total_tokens": data.get("stats", {}).get("total_tokens", 0),
            "tokens_triage": data.get("stats", {}).get("agent_stats", {}).get("triage_agent", {}).get("total_tokens", 0),
            "tokens_planner": data.get("stats", {}).get("agent_stats", {}).get("planner_agent", {}).get("total_tokens", 0),
            "tokens_rca_worker": data.get("stats", {}).get("agent_stats", {}).get("rca_agent", {}).get("total_tokens", 0),
            "runs_count_rca": data.get("stats", {}).get("agent_stats", {}).get("rca_agent", {}).get("runs_count", 0),
            "tokens_supervisor": data.get("stats", {}).get("agent_stats", {}).get("supervisor_agent", {}).get("total_tokens", 0),
            "runs_count_supervisor": data.get("stats", {}).get("agent_stats", {}).get("supervisor_agent", {}).get("runs_count", 0),
            "detection": data.get("final_report", {}).get("detection", None),
            "localization": localization_str, 
            "root_cause": data.get("final_report", {}).get("root_cause", None),
            "eval_detection" : data.get("evaluation", {}).get("detection", None),
            "eval_localization" : data.get("evaluation", {}).get("localization", None),
            "eval_rca_score" : data.get("evaluation", {}).get("rca_score", None),
            "eval_rca_motivation" : data.get("evaluation", {}).get("rca_motivation", None),
        }
        
        # Append record to dataframe
        experiments_df = pd.concat([experiments_df, pd.DataFrame([record])], ignore_index=True)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Warning: Error processing {experiment}: {str(e)}")
        continue

In [ ]:
experiments_df

### Load fault types

In [ ]:
FAULTS_CONF_DIR = os.environ.get("FAULTS_CONF_DIR")

faults_conf_files = [f for f in os.listdir(FAULTS_CONF_DIR) if f.endswith('.json')]

print(f"Found {len(faults_conf_files)} fault configuration files:")

for f in faults_conf_files:
    print(f"- {f}")


In [ ]:
fault_target_dict = {}

for f in faults_conf_files:
    with open(os.path.join(FAULTS_CONF_DIR,f), 'r') as file:
        data = json.load(file)
        formatted_key = f"{data["app_name"]} - {data["fault_type"]}"
        fault_target_dict[formatted_key] = data.get("target", None)

fault_target_dict

## Evaluation

In [ ]:
# Import evaluation helpers from repo package
import sys, os

# Ensure we can import from `sre-agent` package
sys.path.append(os.path.abspath(os.path.join(root_dir, 'sre-agent')))

from evaluation.evaluation import (
    evaluate_detection,
    evaluate_localization,
    evaluate_rca_analysis,
    evaluate_experiment,
)

In [ ]:
# Build ground-truth mapping (target, RCA_gt) from faults configs
faults_gt_dict = {}
try:
    for f in faults_conf_files:
        with open(os.path.join(FAULTS_CONF_DIR, f), 'r') as file:
            d = json.load(file)
            key = f"{d['app_name']} - {d['fault_type']}"
            faults_gt_dict[key] = {
                "target": d.get("target", None),
                "RCA_gt": d.get("RCA_gt", "")
            }
    print(f"Loaded GT for {len(faults_gt_dict)} scenarios.")
except Exception as e:
    print(f"[warn] Failed building faults_gt_dict: {e}")

In [ ]:
def build_fault_scenario(row):
    key = f"{row['scenario']} - {row['fault_name']}"
    gt = faults_gt_dict.get(key, {})
    return {
        "target": gt.get("target", None),
        "RCA_gt": gt.get("RCA_gt", "")
    }

def build_report_from_row(row):
    # Convert localization to list for evaluate_experiment compatibility
    loc_val = row['localization']
    if isinstance(loc_val, str):
        loc_list = [s.strip() for s in loc_val.split(',') if s.strip()]
    elif isinstance(loc_val, list):
        loc_list = loc_val
    else:
        loc_list = []
    return {
        "agent_configuration_name": row.get("agent_conf_name", "N/A"),
        "final_report": {
            "detection": bool(row.get("detection", False)),
            "localization": loc_list,
            "root_cause": row.get("root_cause", "") if isinstance(row.get("root_cause", ""), str) else ""
        }
    }

In [ ]:
from typing import Tuple, Optional
from tqdm import tqdm

def backfill_missing_evaluations(limit: Optional[int] = None, dry_run: bool = False) -> Tuple[int, int]:
    """
    For rows where eval_* fields are missing, compute evaluation via evaluate_experiment,
    persist to the result JSON (adds/overwrites the "evaluation" key), and update the DataFrame.

    Args:
        limit: optional maximum number of rows to process.
        dry_run: if True, do not write files; only compute and update the DataFrame in-memory.

    Returns:
        (processed_count, file_updates)
    """
    required_cols = ['eval_detection', 'eval_localization', 'eval_rca_score', 'eval_rca_motivation']
    missing_mask = experiments_df[required_cols].isna().any(axis=1)
    idxs = experiments_df[missing_mask].index.tolist()
    if limit is not None:
        idxs = idxs[:limit]
    
    processed = 0
    updated_files = 0
    
    for idx in tqdm(idxs, desc="Backfilling evaluations", unit="row"):
        row = experiments_df.loc[idx]
        # Build inputs for evaluation
        fault_scenario = build_fault_scenario(row)
        report = build_report_from_row(row)
        
        # Run evaluation (this may call the LLM for RCA scoring)
        evaluation = evaluate_experiment(fault_scenario, report)
        if not isinstance(evaluation, dict):
            print(f"[warn] Row {idx}: evaluate_experiment returned non-dict; skipping.")
            continue
        
        # Update dataframe in-memory
        experiments_df.loc[idx, 'eval_detection'] = evaluation.get('detection', None)
        experiments_df.loc[idx, 'eval_localization'] = evaluation.get('localization', None)
        experiments_df.loc[idx, 'eval_rca_score'] = evaluation.get('rca_score', None)
        experiments_df.loc[idx, 'eval_rca_motivation'] = evaluation.get('rca_motivation', None)
        
        # Persist to file
        if not dry_run:
            file_name = row['experiment_file']
            file_path = os.path.join(EXPERIMENT_DIR, file_name)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                data['evaluation'] = evaluation
                with open(file_path, 'w') as f:
                    json.dump(data, f, indent=2)
                updated_files += 1
            except Exception as e:
                print(f"[warn] Row {idx}: failed writing evaluation to {file_path}: {e}")
        
        processed += 1
    
    print(f"Backfill completed. Processed rows: {processed}. Files updated: {updated_files}.")
    return processed, updated_files


In [ ]:
backfill_missing_evaluations()

In [ ]:
experiments_df

## Data exploration

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create figure and axis
fig, ax = plt.subplots(figsize=(14, 8))

# Get unique fault types for coloring
fault_types = experiments_df['fault_name'].unique()
colors = plt.cm.Accent(np.linspace(0, 1, len(fault_types)))
color_map = {fault: colors[i] for i, fault in enumerate(fault_types)}

# Plot each experiment as a dot
for idx, row in experiments_df.iterrows():
    x = row['execution_time_seconds']
    y = row['total_tokens']
    fault = row['fault_name']
    color = color_map[fault]
    detection = row['detection']
    
    # Plot the dot - filled if detection is True, only border if False
    if detection:
        ax.scatter(x, y, s=100, color=color, alpha=0.7, edgecolors='black', linewidth=0.5)
    else:
        ax.scatter(x, y, s=100, facecolors='none', edgecolors=color, linewidth=2, alpha=0.7)
    
    # Add label with agent configuration on top of the dot
    label = f"{row['agent_id']}"
    ax.annotate(label, (x, y), fontsize=8, ha='center', va='bottom', fontweight='bold', 
                xytext=(0, 7), textcoords='offset points')

# Create legend for fault types
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map[fault], 
                       markersize=8, label=fault) for fault in fault_types]

# Add detection legend
detection_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', 
               markersize=8, label='Detection: True', markeredgecolor='black', markeredgewidth=0.5),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='none', 
               markersize=8, label='Detection: False', markeredgecolor='gray', markeredgewidth=2)
]

# Combine all handles into a single legend
all_handles = handles + detection_handles
ax.legend(handles=all_handles, loc='upper left', fontsize=10)

# Labels and title
ax.set_xlabel('Execution Time (seconds)', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Tokens', fontsize=12, fontweight='bold')
ax.set_title('Hotel reservation scenario', fontsize=14, fontweight='bold')

# Grid
ax.grid(True, alpha=0.3, linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

# Create figure and axis
fig, ax = plt.subplots(figsize=(16, 9))

# Get unique fault types for coloring
fault_types = experiments_df['fault_name'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(fault_types)))
color_map = {fault: colors[i] for i, fault in enumerate(fault_types)}

# Collect points and labels for later adjustment
texts = []

# Plot each experiment as a dot
for idx, row in experiments_df.iterrows():
    x = row['execution_time_seconds']
    y = row['total_tokens']
    fault = row['fault_name']
    color = color_map[fault]
    correct_localization = row['correct_localization']
    
    # Plot the dot - filled if correct_localization is True, X marker if False
    if correct_localization:
        ax.scatter(x, y, s=150, color=color, alpha=0.8, edgecolors='black', linewidth=1.2)
    else:
        ax.scatter(x, y, s=150, marker='x', color=color, linewidth=2.5, alpha=0.8)
    
    # Add label with agent configuration
    label = f"{row['agent_id']}"
    text_obj = ax.text(x, y, label, fontsize=9, ha='center', va='center', fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.4', facecolor='white', edgecolor='gray', alpha=0.85, linewidth=0.8),
                       zorder=10)
    texts.append(text_obj)

# Adjust text positions to avoid overlaps
try:
    adjust_text(texts, arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2', 
                                       lw=0.8, color='gray', alpha=0.6),
                ax=ax, precision=0.1, expand_points=(1.5, 1.5))
except ImportError:
    print("Note: adjustText not installed. Labels may overlap. Install with: pip install adjustText")

# Create legend for fault types
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map[fault], 
                       markersize=10, label=fault, markeredgecolor='black', markeredgewidth=1) 
           for fault in sorted(fault_types)]

# Add localization legend
localization_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', 
               markersize=10, label='Correct Localization: True', markeredgecolor='black', markeredgewidth=1),
    plt.Line2D([0], [0], marker='x', color='gray', linestyle='None',
               markersize=10, label='Correct Localization: False', markeredgewidth=2.5)
]

# Combine all handles into a single legend
all_handles = handles + localization_handles
ax.legend(handles=all_handles, loc='upper left', fontsize=11, framealpha=0.95, edgecolor='black')

# Labels and title
ax.set_xlabel('Execution Time (seconds)', fontsize=14, fontweight='bold', color='#2c3e50')
ax.set_ylabel('Total Tokens', fontsize=14, fontweight='bold', color='#2c3e50')
ax.set_title('Localization Accuracy: Execution Time vs Token Usage\n(by Fault Type and Agent Configuration)', 
             fontsize=16, fontweight='bold', pad=20, color='#2c3e50')

# Grid and background
ax.set_facecolor('#f8f9fa')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, color='#bdc3c7')
ax.set_axisbelow(True)

# Spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#bdc3c7')
ax.spines['bottom'].set_color('#bdc3c7')
ax.spines['left'].set_linewidth(1.2)
ax.spines['bottom'].set_linewidth(1.2)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate distributions first
global_dist = experiments_df['correct_localization'].value_counts(normalize=True).sort_index(ascending=False)
fault_type_dist = experiments_df.groupby('fault_name')['correct_localization'].mean().sort_values(ascending=False)
agent_config_dist = experiments_df.groupby(['rca_tasks_per_iteration', 'max_tool_calls'])['correct_localization'].mean().sort_values(ascending=False)

import matplotlib.ticker as mtick
import seaborn as sns

# Set modern style
plt.style.use('default')
sns.set_palette("husl")

# Configure matplotlib for modern look
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['font.family'] = 'sans-serif'

# ============ GLOBAL DISTRIBUTION ============
fig, ax = plt.subplots(figsize=(8, 6))
colors_global = ['#2ecc71', '#e74c3c']
bars = ax.bar(['Correct', 'Incorrect'], [global_dist.get(True, 0), global_dist.get(False, 0)], 
              color=colors_global, edgecolor='white', linewidth=3, alpha=0.9, width=0.6)

ax.set_title('Localization Accuracy - Overall Results', fontsize=18, fontweight='bold', pad=25, color='#2c3e50')
ax.set_ylabel('Proportion', fontsize=14, fontweight='bold', color='#34495e')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylim(0, max(global_dist.values) * 1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#bdc3c7')
ax.spines['bottom'].set_color('#bdc3c7')
ax.spines['left'].set_linewidth(1)
ax.spines['bottom'].set_linewidth(1)
ax.grid(axis='y', linestyle='--', alpha=0.2, linewidth=0.8, color='#bdc3c7')
ax.set_axisbelow(True)

# Add value labels
total = experiments_df.shape[0]
for i, (label, bar) in enumerate(zip(['Correct', 'Incorrect'], bars)):
    value = bar.get_height()
    count = int(value * total)
    ax.text(bar.get_x() + bar.get_width()/2, value + 0.045, f'{value*100:.1f}%\n({count}/{total})', 
            ha='center', va='bottom', fontsize=13, fontweight='bold', color='#2c3e50')

plt.tight_layout()
plt.show()

# ============ BY FAULT TYPE ============
fig, ax = plt.subplots(figsize=(13, 7))
colors_fault = plt.cm.RdYlGn(np.linspace(0.3, 0.8, len(fault_type_dist)))
bars = ax.barh(range(len(fault_type_dist)), fault_type_dist.values, 
               color=colors_fault, edgecolor='white', linewidth=2.5, alpha=0.9)

ax.set_yticks(range(len(fault_type_dist)))
ax.set_yticklabels(fault_type_dist.index, fontsize=12, fontweight='bold', color='#2c3e50')
ax.set_xlabel('Localization Accuracy Rate', fontsize=14, fontweight='bold', color='#34495e')
ax.set_title('Localization Accuracy by Fault Type', fontsize=18, fontweight='bold', pad=25, color='#2c3e50')
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlim(0, 1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#bdc3c7')
ax.spines['bottom'].set_color('#bdc3c7')
ax.spines['left'].set_linewidth(1)
ax.spines['bottom'].set_linewidth(1)
ax.grid(axis='x', linestyle='--', alpha=0.2, linewidth=0.8, color='#bdc3c7')
ax.set_axisbelow(True)

# Add value labels
for i, (fault, rate) in enumerate(fault_type_dist.items()):
    total_fault = (experiments_df['fault_name'] == fault).sum()
    correct_fault = ((experiments_df['fault_name'] == fault) & (experiments_df['correct_localization'])).sum()
    ax.text(rate + 0.03, i, f'{rate*100:.1f}% ({correct_fault}/{total_fault})', 
            va='center', fontsize=11, fontweight='bold', color='#2c3e50')

plt.tight_layout()
plt.show()

# ============ BY AGENT CONFIGURATION ============
fig, ax = plt.subplots(figsize=(11, 6))
config_labels = [f"P{p}-T{t}" for p, t in agent_config_dist.index]
colors_config = plt.cm.viridis(np.linspace(0.2, 0.9, len(agent_config_dist)))
bars = ax.bar(range(len(agent_config_dist)), agent_config_dist.values,
              color=colors_config, edgecolor='white', linewidth=2.5, alpha=0.9, width=0.65)

ax.set_xticks(range(len(agent_config_dist)))
ax.set_xticklabels(config_labels, fontsize=12, fontweight='bold', color='#2c3e50', rotation=45, ha='right')
ax.set_ylabel('Localization Accuracy Rate', fontsize=14, fontweight='bold', color='#34495e')
ax.set_title('Localization Accuracy by Agent Configuration', fontsize=18, fontweight='bold', pad=25, color='#2c3e50')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylim(0, max(agent_config_dist.values) * 1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#bdc3c7')
ax.spines['bottom'].set_color('#bdc3c7')
ax.spines['left'].set_linewidth(1)
ax.spines['bottom'].set_linewidth(1)
ax.grid(axis='y', linestyle='--', alpha=0.2, linewidth=0.8, color='#bdc3c7')
ax.set_axisbelow(True)

# Add value labels
for i, ((p, t), rate) in enumerate(agent_config_dist.items()):
    mask = (experiments_df['rca_tasks_per_iteration'] == str(p)) & (experiments_df['max_tool_calls'] == str(t))
    total_cfg = mask.sum()
    correct_cfg = (mask & experiments_df['correct_localization']).sum()
    ax.text(i, rate + 0.048, f'{rate*100:.1f}%\n({correct_cfg}/{total_cfg})', 
            ha='center', va='bottom', fontsize=10, fontweight='bold', color='#2c3e50')

plt.tight_layout()
plt.show()


In [ ]:
agent_tokens = experiments_df[[
    'tokens_triage',
    'tokens_planner', 
    'tokens_rca_worker',
    'tokens_supervisor'
]].sum()

fig, ax = plt.subplots(figsize=(10, 6))
agent_tokens.plot(kind='bar', ax=ax, color=['#FF9999', '#66B2FF', '#99FF99', '#FFD700'])
ax.set_ylabel('Total Tokens', fontsize=12, fontweight='bold')
ax.set_title('Token Distribution Across Agents', fontsize=14, fontweight='bold')
ax.set_xticklabels(['Triage', 'Planner', 'RCA Worker', 'Supervisor'], rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
iterations = experiments_df['rca_tasks_per_iteration'].unique()
colors = plt.cm.viridis(np.linspace(0, 1, len(iterations)))

for i, iter_val in enumerate(sorted(iterations)):
    mask = experiments_df['rca_tasks_per_iteration'] == iter_val
    ax.scatter(experiments_df[mask]['execution_time_seconds'], 
              experiments_df[mask]['total_tokens'],
              label=f'RCA Tasks: {int(iter_val)}',
              s=100, alpha=0.7, color=colors[i])

ax.set_xlabel('Execution Time (seconds)', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Tokens', fontsize=12, fontweight='bold')
ax.set_title('Impact of RCA Tasks per Iteration on Performance', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()